# Phase 2: Model Training and Experimentation

This notebook will focus on training and optimizing our model. Our objectives include
1. Experiment tracking with MLflow
2. Cross-validation
3. Hyperparameter optimization
4. Model evaluation and selection
5. Model versioning and registration

In [1]:
%pip install mlflow optuna

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import joblib
import optuna
import xgboost as xgb
from pathlib import Path

# import custom transformers
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from models.custom_transformers import FeatureEngineer, LogTransformer

# Configure MLflow
# mlflow.set_tracking_uri("http://localhost:8000")
EXPERIMENT_NAME = "ames-housing-price-prediction"
mlflow.set_experiment(EXPERIMENT_NAME)

# Load the preprocessor
# if these errors out, try running phase1_eda.ipynb first
models_dir = "../models"
feature_preprocessor = joblib.load(f'{models_dir}/feature_preprocessor.joblib')
target_transformer = joblib.load(f'{models_dir}/target_transformer.joblib')

print("Loading data...")
train_data = pd.read_csv('../data/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('../data/house-prices-advanced-regression-techniques/test.csv')
print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

/Users/mak/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025/11/05 00:03:55 INFO mlflow.tracking.fluent: Experiment with name 'ames-housing-price-prediction' does not exist. Creating a new experiment.


Loading data...
Train shape: (1460, 81)
Test shape: (1459, 80)


In [5]:
X = train_data.drop('SalePrice', axis=1)
y = train_data['SalePrice']

# apply pipelines/transformations
X_processed = feature_preprocessor.transform(X)
y_processed = target_transformer.transform(y)

# Split data into train and validation sets
# use _val to prevent confusion between the test dataset
X_train, X_val, y_train, y_val = train_test_split(
    X_processed, y_processed, 
    test_size=0.2, 
    random_state=42
)

# Apply preprocessing
print("Applying preprocessing pipeline...")
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)

Applying preprocessing pipeline...
X_train shape: (1168, 242)
X_val shape: (292, 242)


## Baseline Model Development

Let's start with a simple baseline model using XGBoost with default parameters. This will give us a reference point for further improvements.

In [6]:
def evaluate_model(model, X, y, prefix=''):
    y_pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    return {
        f'{prefix}rmse': rmse,
        f'{prefix}mae': mae,
        f'{prefix}r2': r2
    }

In [7]:
# for training baseline XGBoost model
with mlflow.start_run(run_name="xgboost-baseline"):
    # create model with default parameters
    model = xgb.XGBRegressor(
        random_state=42,
        n_jobs=-1
    )
    
    # train model
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_val, y_val)],
        verbose=100
    )
    
    train_metrics = evaluate_model(model, X_train, y_train, prefix='train_')
    val_metrics = evaluate_model(model, X_val, y_val, prefix='val_')
    
    mlflow.log_params(model.get_params())
    mlflow.log_metrics({**train_metrics, **val_metrics})
    mlflow.xgboost.log_model(model, "model")
    
    print("\nTraining Metrics:")
    for metric, value in train_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    print("\nValidation Metrics:")
    for metric, value in val_metrics.items():
        print(f"{metric}: {value:.4f}")

# Save the baseline model
joblib.dump(model, f'{models_dir}/baseline_xgboost.joblib')
print(f"\nBaseline model saved to {models_dir}/baseline_xgboost.joblib")

[0]	validation_0-rmse:0.33810


[99]	validation_0-rmse:0.15271


2025/11/04 17:50:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1115: UserWarning: [17:50:38] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/11/04 17:50:40 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/04 17:50:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Training Metrics:
train_rmse: 0.0076
train_mae: 0.0055
train_r2: 0.9996

Validation Metrics:
val_rmse: 0.1527
val_mae: 0.1064
val_r2: 0.8750
🏃 View run xgboost-baseline at: http://localhost:8000/#/experiments/1/runs/802977bc39af45ddbd8e554593769deb
🧪 View experiment at: http://localhost:8000/#/experiments/1

Baseline model saved to ../models/baseline_xgboost.joblib


## Hyperparameter Optimization with Optuna

Now that we have a baseline model, let's use Optuna to find better hyperparameters for our XGBoost model. We'll define an objective function that Optuna will optimize using cross-validation scores.

In [8]:
# objective function for Optuna
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'random_state': 42
    }
    
    # get model with suggested parameters
    model = xgb.XGBRegressor(**params)
    
    # cross-validation
    cv_scores = cross_val_score(
        model, 
        X_train, 
        y_train, 
        cv=5, 
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    
    # mean negative RMSE (Optuna minimizes objective)
    return -cv_scores.mean()

# Create and run Optuna study
study = optuna.create_study(direction='minimize')
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-11-04 17:50:40,988] A new study created in memory with name: no-name-16d042df-d7fc-473d-813d-947a4dd438a1
Best trial: 47. Best value: 0.124093: 100%|██████████| 50/50 [00:30<00:00,  1.66it/s]

Best trial:
  Value:  0.12409305778478182
  Params: 
    max_depth: 3
    learning_rate: 0.03308285170291822
    n_estimators: 834
    min_child_weight: 4
    subsample: 0.7286386786349668
    colsample_bytree: 0.6201182915351288
    reg_alpha: 1.3640537686252489e-07
    reg_lambda: 2.3734191995955346e-08


In [9]:
# for training the optimized model
with mlflow.start_run(run_name="xgboost-optimized"):
    # get model with best parameters
    best_params = study.best_params
    best_params['random_state'] = 42
    model = xgb.XGBRegressor(**best_params)
    
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_val, y_val)],
        verbose=100
    )
    
    train_metrics = evaluate_model(model, X_train, y_train, prefix='train_')
    val_metrics = evaluate_model(model, X_val, y_val, prefix='val_')
    
    mlflow.log_params(model.get_params())
    mlflow.log_metrics({**train_metrics, **val_metrics})
    mlflow.xgboost.log_model(model, name="model")
    
    print("\nTraining Metrics:")
    for metric, value in train_metrics.items():
        print(f"{metric}: {value:.4f}")
    
    print("\nValidation Metrics:")
    for metric, value in val_metrics.items():
        print(f"{metric}: {value:.4f}")

# save optimized model
joblib.dump(model, '../models/optimized_xgboost.joblib')
print("\nOptimized model saved to ../models/optimized_xgboost.joblib")

[0]	validation_0-rmse:0.42364
[100]	validation_0-rmse:0.15476
[200]	validation_0-rmse:0.13882
[300]	validation_0-rmse:0.13435
[400]	validation_0-rmse:0.13270
[500]	validation_0-rmse:0.13262
[600]	validation_0-rmse:0.13259
[700]	validation_0-rmse:0.13275
[800]	validation_0-rmse:0.13326
[833]	validation_0-rmse:0.13342


/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1115: UserWarning: [17:51:12] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/11/04 17:51:14 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/04 17:51:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Training Metrics:
train_rmse: 0.0574
train_mae: 0.0420
train_r2: 0.9784

Validation Metrics:
val_rmse: 0.1334
val_mae: 0.0894
val_r2: 0.9046
🏃 View run xgboost-optimized at: http://localhost:8000/#/experiments/1/runs/5f17ca27d7dd4c32a242aa61282e04c9
🧪 View experiment at: http://localhost:8000/#/experiments/1

Optimized model saved to ../models/optimized_xgboost.joblib


## Model Comparison

Let's compare the performance of our baseline and optimized models to see the improvement from hyperparameter optimization.

In [10]:
baseline_model = joblib.load('../models/baseline_xgboost.joblib')

# compare models on validation set
baseline_metrics = evaluate_model(baseline_model, X_val, y_val)
optimized_metrics = evaluate_model(model, X_val, y_val)

print("Baseline Model Metrics:")
for metric, value in baseline_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nOptimized Model Metrics:")
for metric, value in optimized_metrics.items():
    print(f"{metric}: {value:.4f}")

improvement = (baseline_metrics['rmse'] - optimized_metrics['rmse']) / baseline_metrics['rmse'] * 100
print(f"\nRMSE Improvement: {improvement:.2f}%")

Baseline Model Metrics:
rmse: 0.1527
mae: 0.1064
r2: 0.8750

Optimized Model Metrics:
rmse: 0.1334
mae: 0.0894
r2: 0.9046

RMSE Improvement: 12.63%


In [11]:
# Compose the preprocessor and trained model into a single sklearn-compatible object
# and use TransformedTargetRegressor so the target transformer is applied automatically.
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

# Build an sklearn Pipeline that first applies the feature preprocessor then the model.
# Note: feature_preprocessor and model are expected to be already fitted objects.
skl_pipeline = Pipeline([
    ("preprocessor", feature_preprocessor),
    ("regressor", model),
])

# Wrap with TransformedTargetRegressor so predictions are returned on the original scale.
full_pipeline = TransformedTargetRegressor(regressor=skl_pipeline, transformer=target_transformer)
full_pipeline.fit(X, y)

# Save the composed object (sklearn pipeline or fallback wrapper)
models_path = Path(models_dir)
models_path.mkdir(parents=True, exist_ok=True)
joblib.dump(full_pipeline, models_path / "full_pipeline.joblib")
print(f"Saved full pipeline to {models_path / 'full_pipeline.joblib'}")

# Example: predict on a single row from the test set (keeps original columns)
example_row = test_data.iloc[[0]]  # DataFrame with one row
pred = full_pipeline.predict(example_row)
print("Example prediction (SalePrice):", pred)


Saved full pipeline to ../models/full_pipeline.joblib
Example prediction (SalePrice): [121776.33]
